## The Transportation Problem

The transportation problem is a special case of the minimal-cost network-flow problem. There are only source and sink nodes without any intermediate ones.

$N_S$ denotes the set of supply nodes and $N_D$ - the set of demand nodes. $N_S \cap N_D = \emptyset$ and $N_S \cup N_D = N$.

$$\text{min} \sum_{i \in{N_S}} \sum_{j \in{N_D}} c_{ij} x_{ij}$$

$\text{subject to}$

$$\sum_{j \in{N_D}} x_{ij} = s_i \quad \forall{i} \in{N_S}$$

$$\sum_{i \in{N_D}} x_{ij} = d_j \quad \forall{j} \in{N_D}$$

$$x_{ij} \ge 0 \quad \forall{i} \in{N_S} ,j \in{N_D}$$

where $c_{ij}$ is the unit transportation cost from supply node $i$ to sink node $j$, and $s_i$ and $d_j$ are *nonnegative* constants for the supply and demand amounts, respectively.

It is assumed that $\sum_{i \in{N_S}} s_i = \sum_{j \in{N_D}} d_j$.

In [1]:
data = readcsv("transportation_problem.csv")

4×5 Array{Any,2}:
   ""  ""         15           12          13      
   ""  ""           "Chicago"    "Denver"    "Erie"
 15    "Austin"   10            7           9      
 25    "Buffalo"   4            9           8      

In [10]:
supply_nodes = collect(data[3:end, 2])
s = collect(data[3:end, 1])

demand_nodes = collect(data[2, 3:end])
d = collect(data[1, 3:end])

c = data[3:end, 3:end]

s_dict = Dict(zip(supply_nodes, s))
d_dict = Dict(zip(demand_nodes, d))

c_dict = Dict()
for i in 1:length(supply_nodes)
    for j in 1:length(demand_nodes)
        c_dict[supply_nodes[i], demand_nodes[j]] = c[i, j]
    end
end

c_dict

Dict{Any,Any} with 6 entries:
  ("Austin", "Chicago")  => 10
  ("Buffalo", "Denver")  => 9
  ("Austin", "Erie")     => 9
  ("Buffalo", "Chicago") => 4
  ("Austin", "Denver")   => 7
  ("Buffalo", "Erie")    => 8

## Solution

In [13]:
using JuMP, Clp

model = Model(solver=ClpSolver())

@variable(model, x[supply_nodes, demand_nodes] >= 0)

@objective(model, Min, sum(c_dict[i, j] * x[i, j] for i in supply_nodes for j in demand_nodes))
                
for i in supply_nodes
    @constraint(model, sum(x[i, j] for j in demand_nodes) == s_dict[i])
end
                
for j in demand_nodes
    @constraint(model, sum(x[i, j] for i in supply_nodes) == d_dict[j])
end
                
model 

Minimization problem with:
 * 5 linear constraints
 * 6 variables
Solver is ClpMathProg

In [18]:
status = solve(model)

@printf "Objective: %.2f (%s)" getobjectivevalue(model) status
@printf "\n\nSolution:"

getvalue(x)

Objective: 251.00 (Optimal)

Solution:

x: 2 dimensions:
[ Austin,:]
  [ Austin,Chicago] = 0.0
  [ Austin, Denver] = 12.0
  [ Austin,   Erie] = 3.0
[Buffalo,:]
  [Buffalo,Chicago] = 15.0
  [Buffalo, Denver] = 0.0
  [Buffalo,   Erie] = 10.0